# Extracting FF14 Purchase History from Universalis.app 

In [1]:
import requests
import re
import json
import csv
import time
import datetime

### Generate links for item & transactions for each server

In [2]:
# Link structure: 'https://universalis.app/api/history/Mateus/23893?entries=1800'
item_ID = 26108
transactions = 5
URL = 'https://universalis.app/api/history/'
servers = ['Balmung','Brynhildr','Coeurl','Diabolos','Goblin','Marlboro','Mateus','Zalera']

for server in servers:
    print(URL+str(server)+'/'+str(item_ID)+'?entries='+str(transactions))

https://universalis.app/api/history/Balmung/26108?entries=5
https://universalis.app/api/history/Brynhildr/26108?entries=5
https://universalis.app/api/history/Coeurl/26108?entries=5
https://universalis.app/api/history/Diabolos/26108?entries=5
https://universalis.app/api/history/Goblin/26108?entries=5
https://universalis.app/api/history/Marlboro/26108?entries=5
https://universalis.app/api/history/Mateus/26108?entries=5
https://universalis.app/api/history/Zalera/26108?entries=5


### Preview json

In [3]:
req = requests.get('https://universalis.app/api/history/Mateus/26108?entries=2')
sales = json.loads(req.text)
sales

{'itemID': 26108,
 'worldID': 37,
 'lastUploadTime': 1655323242875,
 'entries': [{'hq': True,
   'pricePerUnit': 45990,
   'quantity': 1,
   'buyerName': 'Sirius Vonlinden',
   'timestamp': 1655238872},
  {'hq': True,
   'pricePerUnit': 45990,
   'quantity': 1,
   'buyerName': 'Custom Preset',
   'timestamp': 1655154627}],
 'stackSizeHistogram': {'1': 613},
 'stackSizeHistogramNQ': {'1': 17},
 'stackSizeHistogramHQ': {'1': 596},
 'regularSaleVelocity': 1.1428572,
 'nqSaleVelocity': 0,
 'hqSaleVelocity': 1.1428572,
 'worldName': 'Mateus'}

### Troubleshooting 1: Missing 'entries':'buyerName'

In [4]:
req = requests.get('https://universalis.app/api/history/Mateus/26108?entries=100')
sales = json.loads(req.text)
sales['entries'][47:50]

[{'hq': True,
  'pricePerUnit': 29990,
  'quantity': 1,
  'buyerName': 'Saelstine Ariesvyn',
  'timestamp': 1653265071},
 {'hq': True, 'pricePerUnit': 29989, 'quantity': 1, 'timestamp': 1653184950},
 {'hq': True, 'pricePerUnit': 45000, 'quantity': 1, 'timestamp': 1653067347}]

### Troubleshooting 2: Servers with 0 purchases for item of interest

In [5]:
req = requests.get('https://universalis.app/api/history/Marlboro/26108?entries=10')
sales = json.loads(req.text)
sales

{'type': 'https://tools.ietf.org/html/rfc7231#section-6.5.4',
 'title': 'Not Found',
 'status': 404,
 'traceId': '00-1938146da9abd646bef2f1ef505d66c7-36132ace2a7051ff-00'}

### Loop through server links and jsons

In [6]:
def purchase_data(item_ID, transactions, servers):
    URL = 'https://universalis.app/api/history/'
    cross_server_data = []
    
    for server in servers:
        req = requests.get(URL+str(server)+'/'+str(item_ID)+'?entries='+str(transactions))
        sales = json.loads(req.text)
        output = []
        
        # add exception-handling for servers with 0 transactions
        try:
            for i in sales['entries']:
                row = [sales['itemID'], sales['worldName'], sales['regularSaleVelocity'],sales['nqSaleVelocity'], 
                       sales['hqSaleVelocity'], i['pricePerUnit'], i['quantity'], i['hq'],
                       datetime.datetime.utcfromtimestamp(i['timestamp']).strftime("%m/%d/%Y,%H:%M:%S")]

                # add exception-handling for entries where buyerName didn't exist
                try:
                    row.insert(len(row)-1, i['buyerName'])
                except:
                    row.insert(len(row)-1, "na")

                output.append(row)
        except:
            pass
        
        cross_server_data.append(output)
        
    return cross_server_data

### Write to csv

In [7]:
item_ID = 26108
transactions = 100
servers = ['Balmung','Brynhildr','Coeurl','Diabolos','Goblin','Marlboro','Mateus','Zalera']

with open('Marketboard_Purchase_History.csv','w',  encoding='utf-8', newline="") as f:
    fr = csv.writer(f)
    #Write Header
    fr.writerow(('itemID', 'Server', 'regularSaleVelocity','nqSalesVelocity',
                 'HQSalesVelocity', 'Price', 'Quantity','HQ', 'buyerName','Timestamp'))

    #Write rows
    for i in range(len(purchase_data(item_ID, transactions, servers))):
        for row in purchase_data(item_ID, transactions, servers)[i]:
            fr.writerow(row)

In [8]:
#Preview csv
with open('Marketboard_Purchase_History.csv', 'r', encoding='utf-8') as f:
    fr = f.readlines()
fr[0:10]

['itemID,Server,regularSaleVelocity,nqSalesVelocity,HQSalesVelocity,Price,Quantity,HQ,buyerName,Timestamp\n',
 '26108,Balmung,0.42857143,0,0.42857143,28494,1,True,Beni\'to Camelas,"06/15/2022,17:53:11"\n',
 '26108,Balmung,0.42857143,0,0.42857143,55440,1,True,The Hellwalker,"06/14/2022,00:48:49"\n',
 '26108,Balmung,0.42857143,0,0.42857143,55470,1,True,Erika Koutari,"06/10/2022,15:06:27"\n',
 '26108,Balmung,0.42857143,0,0.42857143,55490,1,True,Katja Starfall,"06/05/2022,20:44:05"\n',
 '26108,Balmung,0.42857143,0,0.42857143,59990,1,True,Katja Starfall,"06/05/2022,03:49:17"\n',
 '26108,Balmung,0.42857143,0,0.42857143,59990,1,True,Y\'nahra Rahl,"06/01/2022,09:18:23"\n',
 '26108,Balmung,0.42857143,0,0.42857143,59990,1,True,Sorivian Evenfall,"05/30/2022,04:17:34"\n',
 '26108,Balmung,0.42857143,0,0.42857143,29500,1,True,Emissary Themis,"05/29/2022,20:23:37"\n',
 '26108,Balmung,0.42857143,0,0.42857143,30000,1,True,Hermund Hammerfist,"05/28/2022,04:58:02"\n']